In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
import time
import random



def check_data(data):
    try:
        technicalSheet = data['props']['pageProps']['data']['technicalSheet']
        longitude = data['props']['pageProps']['data']['longitude']
        latitude = data['props']['pageProps']['data']['latitude']
        stratum = technicalSheet[0]['value'] if technicalSheet[0]['value'] is not None  else 0
        bathrooms = technicalSheet[3]['value'] if technicalSheet[3]['value'] is not None  else 0
        constructed_area = float(technicalSheet[4]['value'].replace('  m2', '')) if technicalSheet[4]['value'] is not None  else 0
        house_age = technicalSheet[6]['value'] 
        bedrooms = technicalSheet[7]['value'] if technicalSheet[7]['value'] is not None  else 0
        parking = technicalSheet[8]['value'] if technicalSheet[8]['value'] is not None  else 0
        administration = float(technicalSheet[9]['value'].replace(',', '')) if technicalSheet[9]['value'] is not None  else 0
        floor = technicalSheet[10]['value'] if technicalSheet[10]['value'] is not None  else 0
        rent_price = data['props']['pageProps']['data']['price']['amount'] 
        rent_price = rent_price if rent_price is not None else 0
        facilities = [i['name'] for i in data['props']['pageProps']['data']['facilities']]
        
        if house_age is not None and house_age != 'menor a 1 año' and house_age != 'más de 30 años':
            age_range = house_age.replace(' años', '').split(' a ')
            age_range = list(map(int, age_range))
            house_age = int(sum(age_range)/len(age_range))
        else: 
            if house_age == 'más de 30 años':
                house_age = 31
            else:
                house_age = 0
        
        return [longitude, latitude, stratum, bathrooms, constructed_area, house_age, bedrooms, parking, administration, floor, rent_price, facilities]
    
    except KeyError as e:
        print(f'Error: clave {e} no encontrada en los datos JSON.')
        return [None]*12

In [5]:
df = pd.DataFrame(columns=['longitude', 'latitude', 'stratum', 'bathrooms', 'constructed_area', 'house_age', 'bedrooms', 'parking', 'administration', 'floor', 'rent_price', 'facilities'])
n_pages = 15
next_page = ''

headers = {
    # Encabezado User-Agent que indica: navegador compatible con Mozilla, en Windows 10 (64-bit), usando el motor 
    # WebKit como Chrome v91, con referencias a Gecko y Safari para compatibilidad
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

session = requests.Session()
# itera sobre las 200 primeras paginas de apartamentos en arriendo en finca raiz
for i in range(1, n_pages+1):
    # Se usa try para prevenir errores y no suspenda la consulta
    try:
        url = 'https://www.fincaraiz.com.co/arriendo/apartamentos/bogota/bogota-dc' + next_page
        html_text = session.get(url, headers=headers).text
    except requests.exceptions.RequestException as e:
        print(f'Error al conectar con la pagina {url}\n {e}')
        continue
    
    #la siguiente linea pausa la ejecucion del codigo durante un tiempo aleatorio entre 1 y 4 seg para evitar que la pagina bloquee las consultas
    time.sleep(random.uniform(1, 4))
    #El archivo html se convierte a archivo lxml que una estructura mas eficiente para buscar en el DOM
    soup1 = BeautifulSoup(html_text, 'lxml')
    #Esta linea busca todos los links con la clase 'lc-cardCover' que son las cards de los apartamentos
    cards = soup1.find_all('a', class_ = 'lc-cardCover')

    #Se itera sobre todas las cards obtenidas y se extrae el atributo href para hacer la consulta al apartamento especifico
    for j in cards:
        try:
            url_apartment_features = 'https://www.fincaraiz.com.co'+j.get('href')
            apartment_features = session.get(url_apartment_features, headers=headers).text
            soup_apartment_f = BeautifulSoup(apartment_features, 'lxml')
            script_data = soup_apartment_f.find('script', id='__NEXT_DATA__')
            if script_data:
                #Se convierte script_data de JSON a un diccionario de Python para acceder facilmente a los datos
                json_data = json.loads(script_data.string)
                #Se extraen los features con check_data y se agregan al dataframe
                df.loc[len(df)] = check_data(json_data)
            else:
                print(f"No se encontró el script en {url_apartment_features}")
            #Nuevamente se demora la consulta para evitar bloqueos
            time.sleep(random.uniform(1,4))
        except requests.exceptions.RequestException as e:
            print(f'Error al conectar con la card {url_apartment_features}\n {e}')
            continue
    #Cuando se consultan todas las cards se cambia de pagina agregando 'pagina' y el consecutivo numero
    next_page = 'pagina' + str(i+1)
    print(next_page)
df.to_csv('Real_State_Bogota.csv', index=False)
print(df)

pagina2
pagina3
pagina4
pagina5
pagina6
pagina7
pagina8
pagina9
pagina10
pagina11
pagina12
pagina13
pagina14
pagina15
pagina16
     longitude  latitude  stratum  bathrooms  constructed_area  house_age  \
0   -74.053660  4.660556        6          4             237.0         23   
1   -74.046431  4.679071        5          1              28.0          4   
2   -74.048398  4.700991        5          2              97.0         12   
3   -74.045721  4.671554        6          4             167.0         23   
4   -74.050727  4.700429        5          2              78.0         23   
..         ...       ...      ...        ...               ...        ...   
310 -74.029730  4.714144        5          2             117.0         23   
311 -74.067695  4.751620        4          2             120.0         23   
312 -74.057471  4.672217        5          2              93.0         23   
313 -74.091795  4.673456        4          1              53.0          0   
314 -74.043066  4.689606  

In [6]:
data = pd.read_csv('Real_State_Bogota1.csv')